In [1]:
#ini-iPython

# Executando no diretório principal

In [2]:
import os
os.chdir('../../') #Executando na pasta Visual_Memory
import sys
sys.path.append('./include')
sys.path.append('./src')
sys.path.append('./Workbench/iPython')
#end-iPython

---- Imports ----

In [3]:
# Libraries to be used.
#des-iPython import sys
#des-iPython sys.path.append('../include')
#des-iPython sys.path.append('../src')

In [4]:
# The standard libraries used in the visual memory system.
import os # Class used for communication with the system
import signal # Class used for external interrupt detection.
from math import log10, floor # Functions used for rounding UI

In [5]:
# Used class developed by RoboFEI-HT.
#ini-iPython - Usando Classe Basic como instanciação
os.system("sed -i '/@abstractmethod/d' ./Workbench/iPython/Basic.py")
os.system("sed -i '/__metaclass__/d' ./Workbench/iPython/Basic.py")
#end-iPython
from Basic import * # Standard and abstract class.
from Robots import * # Class responsible for performing robots tracking.
from Landmark import * # Class responsible for performing landmarks tracking.

# Class Behavior
Class responsible for managing the behavior of Visual Memory.
doxygen-iPython

In [6]:
#des-iPython class Behavior(Basic):
#ini-tab -> Iniciando código classe

---- Variables ----

# __posrobot
Variable used to instantiate class responsible for robot speed.
doxygen-iPython

In [7]:
__posrobot = None #self-iPython __posrobot

# parameters
Variable used to instantiate class responsible for robot speed.
doxygen-iPython

In [8]:
parameters = None #self-iPython parameters

In [9]:
#ini-iPython

# ------- Lista de funções herdadas -------



# ------- Lista variaveis -------

#self-iPython me
#self-iPython land
#self-iPython robots

#self-iPython __newrobots
#self-iPython args

# ------- Lista de funções -------

#self-iPython run
#self-iPython readDataLandmarks
#self-iPython readDataRobots
#self-iPython distributeDataRobots
#self-iPython readDataBall
#self-iPython end
#self-iPython killedProcess
#self-iPython signal_term_handler
#self-iPython printPreviousLine
#self-iPython updatingScreen
#self-iPython roundUI

#end-iPython - Fim listas

In [10]:
#eself-iPython signal_term_handler
#edes-iPython def signal_term_handler(self, signal, frame):
def signal_term_handler(signal, frame):
    raise VisualMemoryException(3, '') #self-iPython signal_term_handler

In [11]:
#eself-iPython killedProcess
#edes-iPython def killedProcess(self):
def killedProcess( ):
    signal.signal(signal.SIGTERM, signal_term_handler) #self-iPython killedProcess

# Constructor Class
doxygen-iPython

In [12]:
#ini-iPython
class argumentos:
    numberrobots = 1
    executionperiod = 1000
    debug = True
a = argumentos()
#end-iPython
#des-iPython def __init__(self, a):
#ini-tab -> Criando construtor
killedProcess( ) # Function to monitor external process kill.
args = a # Input arguments.

#edes-iPython super(Behavior, self).__init__("Settings", "Visual Memory")
self = Basic("Settings", "Visual Memory")

# Creating default values and reading values
parameters = {
    "number_robots": 8,
    "execution_period_ms": 100,
    "weight_robot": 0.6,
}
parameters = self._conf.readVariables(parameters)

if args.numberrobots != None:
    parameters["number_robots"] = args.numberrobots

if args.executionperiod != None:
    parameters["execution_period_ms"] = args.executionperiod

if parameters["number_robots"] < 0 or parameters["number_robots"] > 22:
    raise VisualMemoryException(1, parameters["number_robots"])

# Creating objects to be tracking
me = Speeds( )
land = Landmark(me)

# Support Variables
robots = []
__newrobots = []
__posrobot = [0, 0]

for i in xrange(parameters["number_robots"]-1):
    __newrobots.append(Robots(me, __posrobot))

TypeError: Can't instantiate abstract class Basic with abstract methods __init__

In [ ]:
#end-tab -> Finalizando construtor

In [ ]:
#ini-iPython

## Teste construtor

In [ ]:
self._end( )
__newrobots
#end-iPython

# readDataLandmarks
Responsible for reading the data coming from the vision system.
doxygen-iPython

In [ ]:
#eself-iPython readDataLandmarks
#edes-iPython def readDataLandmarks(self, old):
def readDataLandmarks(old):
    if self._bkb.read_float("VISION_LAND_TAG") == 1:
        data = [{
                "tag": 1,
                "pos": [self._bkb.read_float("VISION_LAND_X"), self._bkb.read_float("VISION_LAND_Y")],
                "time": self._bkb.read_float("VISION_LAND_TIME"),
                "movement": int(self._bkb.read_float("VISION_LAND_MOV")),
            }]
        
        self._bkb.write_float("VISION_LAND_TAG", 0)
        return old + data
    else:
        return old #self-iPython readDataLandmarks

In [ ]:
#ini-iPython

## Teste readDataLandmarks

In [ ]:
self._bkb.write_float("VISION_LAND_TAG", 1)
self._bkb.write_float("VISION_LAND_X", 2)
self._bkb.write_float("VISION_LAND_Y", 3)
self._bkb.write_float("VISION_LAND_TIME", 4)
self._bkb.write_float("VISION_LAND_MOV", 4)
a = []
a = readDataLandmarks(a)
print "a:", a
a = readDataLandmarks(a)
print "a:", a
#end-iPython

# readDataRobots
.
doxygen-iPython

In [ ]:
#eself-iPython readDataRobots
#edes-iPython def readDataRobots(self, old):
def readDataRobots(old):
    data = []
    for number in xrange(1, parameters["number_robots"]):
#         print "number:", number #debug-iPython
        if self._bkb.read_float("VISION_RB" + str(number).zfill(2) + "_TAG") == 0:
            continue
        
        data.append({
            "tag": self._bkb.read_float("VISION_RB" + str(number).zfill(2) + "_TAG") - 2,
            "pos": [self._bkb.read_float("VISION_RB" + str(number).zfill(2) + "_X"), self._bkb.read_float("VISION_RB" + str(number).zfill(2) + "_Y")],
            "time": self._bkb.read_float("VISION_RB" + str(number).zfill(2) + "_TIME"),
            "movement": int(self._bkb.read_float("VISION_RB" + str(number).zfill(2) + "_MOV")),
        })
        
        self._bkb.write_float("VISION_RB" + str(number).zfill(2) + "_TAG", 0)
    
    data = sorted(old + data, key= lambda k: k["time"])
    
    return data #self-iPython readDataRobots

In [ ]:
#ini-iPython

## Teste readDataRobots

In [ ]:
self._bkb.write_float("VISION_RB01_TAG", 2)
self._bkb.write_float("VISION_RB01_X", 1)
self._bkb.write_float("VISION_RB01_Y", 2)
self._bkb.write_float("VISION_RB01_TIME", 3)
self._bkb.write_float("VISION_RB01_MOV", 3)

a = []
a = readDataRobots(a)
print "a:", a

self._bkb.write_float("VISION_RB02_TAG", 1)
self._bkb.write_float("VISION_RB02_X", 1)
self._bkb.write_float("VISION_RB02_Y", 2)
self._bkb.write_float("VISION_RB02_TIME", 2)
self._bkb.write_float("VISION_RB02_MOV", 2)

a = readDataRobots(a)
print "a:", a
#end-iPython

# distributeDataRobots
.
doxygen-iPython

In [ ]:
#eself-iPython distributeDataRobots
#edes-iPython def distributeDataRobots(self, datarobots):
def distributeDataRobots(datarobots):
    if datarobots == [] or __newrobots + robots == []:
        return
    
    index = 0
    time = -1
    while index < len(datarobots):
        data = datarobots[index]
        print "Dado analisado:", data #debug-iPython
        
        # First input data
        if robots == []:
            print "Primeira leitura" #debug-iPython
            candidates = __newrobots.pop(0)
            candidates.updateThread(data)
            robots.append(candidates)
            datarobots.pop(index)
            print "\n" #debug-iPython
            continue
            
        
        if data["time"] != time:
            opponent = [ robot for robot in robots if robot.timenumber == -1 ]
            indefinite = [ robot for robot in robots if robot.timenumber == 0 ]
            teammate = [ robot for robot in robots if robot.timenumber == 1 ]
            print "Criado listas:" #debug-iPython
            print "opponent:", opponent #debug-iPython
            print "indefinite:", indefinite #debug-iPython
            print "teammate:", teammate #debug-iPython
            time = data["time"]

        
        # Calculates the similarity of the data with the objects
        #  Saving position
        __posrobot[0], __posrobot[1] = data["pos"]
        
        #  Generating Candidates
        if data["tag"] == -1:
            candidates = opponent + indefinite
        elif data["tag"] == 1:
            candidates = indefinite + teammate
        else:
            candidates = opponent + indefinite + teammate
        print "Candidatos feitos:", candidates #debug-iPython
        
        #  Calculates the similarity
        candidates.sort()
        
        #  Sends the data to the most similar object and run object update
        if (candidates == [] or candidates[0].weight > parameters["weight_robot"]) and __newrobots != []:
            print "Nenhum candidato proximo" #debug-iPython
            candidates = __newrobots.pop(0)
            candidates.updateThread(data)
            robots.append(candidates)
            datarobots.pop(index)
            index -= 1
        else:
            if candidates[0].timenumber == -1:
                print "candidato adversario" #debug-iPython
                opponent.remove(candidates[0])
                candidates[0].updateThread(data)
                datarobots.pop(index)
                index -= 1
            elif candidates[0].timenumber == 1:
                print "cadidato aliado" #debug-iPython
                teammate.remove(candidates[0])
                candidates[0].updateThread(data)
                datarobots.pop(index)
                index -= 1
            else:
                print "candidato sei lá" #debug-iPython
                indefinite.remove(candidates[0])
                candidates[0].updateThread(data)
                datarobots.pop(index)
                index -= 1
        
        print "\n" #debug-iPython
        index += 1 #self-iPython distributeDataRobots

In [ ]:
#ini-iPython

## Teste distributeDataRobots

In [ ]:
data = [{
    "tag": 1,
    "pos": [3, 2],
    "time": 3.0,
    "movement": 0,
},
    {
    "tag": -1,
    "pos": [4, 6],
    "time": 1.0,
    "movement": 0,
},
    {
    "tag": 1,
    "pos": [4, 6],
    "time": 1.0,
    "movement": 0,
},
    {
    "tag": 0,
    "pos": [4, 6],
    "time": 1.0,
    "movement": 0,
},
    {
    "tag": 1,
    "pos": [4, 6],
    "time": 2.0,
    "movement": 0,
}]

data = sorted(data, key= lambda k: k["time"])

distributeDataRobots(data)

#end-iPython

# readDataBall
Responsible for reading the data coming from the vision system.
doxygen-iPython

In [ ]:
#eself-iPython readDataBall
#edes-iPython def readDataBall(self):
def readDataBall(old):
    if self._bkb.read_float("VISION_BALL_TAG") == 1:
        data = [{
                "tag": 1,
                "pos": [self._bkb.read_float("VISION_BALL_X"), self._bkb.read_float("VISION_BALL_Y")],
                "time": self._bkb.read_float("VISION_BALL_TIME"),
                "movement": self._bkb.read_float("VISION_BALL_MOV"),
            }]
        
        self._bkb.write_float("VISION_BALL_TAG", 0)
        return old + data
    else:
        return old #self-iPython readDataBall

In [ ]:
#ini-iPython

## Teste readDataLandmarks

In [ ]:
self._bkb.write_float("VISION_LAND_TAG", 1)
self._bkb.write_float("VISION_LAND_X", 2)
self._bkb.write_float("VISION_LAND_Y", 3)
self._bkb.write_float("VISION_LAND_TIME", 4)
self._bkb.write_float("VISION_LAND_MOV", 5)
a = []
a = readDataLandmarks(a)
print "a:", a
a = readDataLandmarks(a)
print "a:", a
#end-iPython

# printPreviousLine
.
doxygen-iPython

In [ ]:
#eself-iPython printPreviousLine
#edes-iPython def printPreviousLine(self, text, lines=1, back=True):
def printPreviousLine(text, lines=1, back=True):
    for i in xrange(lines):
        print "\033[F\r",
    print text

    if back:
        for i in xrange(lines - 1):
            print "" #self-iPython printPreviousLine

In [ ]:
#ini-iPython

## Teste printPreviousLine

In [ ]:
print "Ola mundo"
print "Como vai você?"
printPreviousLine("Batata", lines=3)
#end-iPython

# roundUI
.
doxygen-iPython

In [ ]:
#eself-iPython roundUI
#edes-iPython def roundUI(self, x, digits = 4):
def roundUI(x, digits = 4):
    try:
        return round(x, (digits - 1)-int(floor(log10(abs(x)))))
    except ValueError:
        return 0 #self-iPython roundUI

# updatingScreen
.
doxygen-iPython

In [ ]:
#eself-iPython updatingScreen
#edes-iPython def updatingScreen(self):
def updatingScreen( ):
    # Update landmark
    if self._bkb.read_float("VISUAL_MEMORY_LAND_LOC") == 0:
        text = "+---+ Landmark +     Não    +--------------+           00000          +      "
    else:
        text = "+---+ Landmark +     Sim    +--------------+           00000          +      "
    
    texts = [
        "| x |          |            |     "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_LAND_X"))).zfill(5) +"    |                          |      ",
        text,
        "| y |          |            |     "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_LAND_Y"))).zfill(5) +"    |                          |      ",
    ]
    if parameters["number_robots"] > 1:
        for i, text in zip(range(4 + (parameters["number_robots"] - 1)*4, 4 + (parameters["number_robots"] - 1)*4 - 3, -1), texts):
            printPreviousLine(text, lines=i)
    else:
        for i, text in zip(range(4, 1, -1), texts):
            printPreviousLine(text, lines=i)
    
    # Update robots
    for robot in xrange(1, parameters["number_robots"]):
        # Update friend robots
        if robot <= (parameters["number_robots"] - 1)/2:
            if self._bkb.read_float("VISUAL_MEMORY_AL_"+ str(robot).zfill(2) +"_LOC") == 0:
                text = "+---+  Aliado  +     Não    +--------------+           "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_AL_"+ str(robot).zfill(2) +"_MAX_VEL"))).zfill(5) +"          +      "
            else:
                text = "+---+  Aliado  +     Sim    +--------------+           "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_AL_"+ str(robot).zfill(2) +"_MAX_VEL"))).zfill(5) +"          +      "

            texts = [
                "| x |          |            |     "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_AL_"+ str(robot).zfill(2) +"_X"))).zfill(5) +"    |                          |      ",
                text,
                "| y |          |            |     "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_AL_"+ str(robot).zfill(2) +"_Y"))).zfill(5) +"    |                          |      ",
            ]
        else:
            if self._bkb.read_float("VISUAL_MEMORY_OP_"+ str(robot - (parameters["number_robots"] - 1)/2).zfill(2) +"_LOC") == 0:
                text = "+---+ Oponente +     Não    +--------------+           "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_OP_"+ str(robot - (parameters["number_robots"] - 1)/2).zfill(2) +"_MAX_VEL"))).zfill(5) +"          +      "
            else:
                text = "+---+ Oponente +     Sim    +--------------+           "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_OP_"+ str(robot - (parameters["number_robots"] - 1)/2).zfill(2) +"_MAX_VEL"))).zfill(5) +"          +      "

            texts = [
                "| x |          |            |     "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_OP_"+ str(robot - (parameters["number_robots"] - 1)/2).zfill(2) +"_X"))).zfill(5) +"    |                          |      ",
                text,
                "| y |          |            |     "+ str(roundUI(self._bkb.read_float("VISUAL_MEMORY_OP_"+ str(robot - (parameters["number_robots"] - 1)/2).zfill(2) +"_Y"))).zfill(5) +"    |                          |      ",
            ]
        
        for i, text in zip(range((parameters["number_robots"] - robot)*4, (parameters["number_robots"] - robot)*4 - 3, -1), texts):
            printPreviousLine(text, lines=i) #self-iPython updatingScreen

In [ ]:
#ini-iPython

## Teste updatingScreen

In [ ]:
updatingScreen( )
#end-iPython

# run
.
doxygen-iPython

In [ ]:
#eself-iPython run
#des-iPython def run(self):
#ini-tab - Função run
# Initiating variables
datalandmarks = []
datarobots = []
databall = []

if args.debug:
    print "\nPeríodo de execução:", parameters["execution_period_ms"], "\tPeso mínimo:", parameters["weight_robot"]*100, "%"
    print "+---+----------+------------+--------------+--------------------------+      "
    print "|   |   Nome   | Localizado | Posição (cm) | Velocidade máxima (cm/s) |      "
    print "+---+----------+------------+--------------+--------------------------+      "
    print "| x |          |            |     00000    |                          |      "
    print "+---+ Landmark +     Não    +--------------+           00000          +      "
    print "| y |          |            |     00000    |                          |      "
    print "+---+----------+------------+--------------+--------------------------+      "
    
    for i in xrange((parameters["number_robots"] - 1)/2):
        print "| x |          |            |     00000    |                          |      "
        print "+---+  Aliado  +     Não    +--------------+           00000          +      "
        print "| y |          |            |     00000    |                          |      "
        print "+---+----------+------------+--------------+--------------------------+      "
    for i in xrange((parameters["number_robots"] - 1)/2, (parameters["number_robots"] - 1)):
        print "| x |          |            |     00000    |                          |      "
        print "+---+ Oponente +     Não    +--------------+           00000          +      "
        print "| y |          |            |     00000    |                          |      "
        print "+---+----------+------------+--------------+--------------------------+      "

#des-iPython while True:
#ini-tab - Função while
try:
    # Start counting time
    start = time.time()
    print "start:", start #debug-iPython

    # Reading data from landmarks
    datalandmarks = readDataLandmarks(datalandmarks)

    if datalandmarks != []:
        # Predict robot speed (me)
        me.update(land.update(datalandmarks.pop(0)))
    else:
        # Predicts only the new landmarks position
        land.predict(movements = 1)

    # Reading robots data
    datarobots = readDataRobots(datarobots)

    # Distribute the data to the robot objects
    distributeDataRobots(datarobots)

    # # Reading ball data
    # databall = readDataBall(databall)

    # # Distribute the data to the ball objects
    # Batata

    # # Doing cleaning and objects (lost objects)
    # Batata

    # # Predicts objects (Now)
    # Batata

    # # Send object for vision screening
    # Batata
    
    # Displaying debug values
    if args.debug:        
        updatingScreen( )
    

    # Wait an instant on the remaining time
    delta = parameters["execution_period_ms"]*1e-3 - time.time() + start
    print "delta:", delta #debug-iPython
    if delta > 0:
        print "Aguardando", delta*1e3, "ms" #debug-iPython
        time.sleep(delta) #self-iPython run
        print "Período finalizado" #debug-iPython
except KeyboardInterrupt:
    os.system('clear') # Cleaning terminal
    print "Keyboard interrupt detected"
    #edes-iPython break
    pass
except VisualMemoryException as e:
    #edes-iPython break
    pass
#end-tab - Função while
#end-tab - Função run

In [ ]:
#ini-iPython

## Teste run

In [ ]:

#end-iPython

# end
.
doxygen-iPython

In [ ]:
#eself-iPython end
#edes-iPython def end(self):
def end( ):
    self._end( )
    kill = robots + __newrobots
    print "Lista de robos:", kill #debug-iPython
    
    for robot in kill:
        print "Finalizando", robot #debug-iPython
        robot.end( )
        print "Finalizado", robot #debug-iPython
    
    land.end( )
    
    #self-iPython end

In [ ]:
#ini-iPython

## Teste end

In [ ]:
end( )
#end-iPython

In [ ]:
#end-tab -> Finalizando classe

In [ ]:
#ini-iPython

# nova_variavel
.
doxygen-iPython

In [ ]:
nova_variavel = None #self-iPython nova_variavel

# nova_funcao
.
doxygen-iPython

In [ ]:
# eself-iPython nova_funcao
# edes-iPython def nova_funcao(self, variaveis_funcao):
def nova_funcao(variaveis_funcao):
    pass # self-iPython nova_funcao

In [ ]:
#ini-iPython

## Teste nova_funcao

In [ ]:

#end-iPython

In [ ]:
#end-iPython